In [1]:
import psycopg2
import pandas as pd
import json
import pyarrow as pa
import pyarrow.parquet as pq
import os

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [12]:
conn = psycopg2.connect(
    host='localhost',
    port=5434,
    user='airflow',
    password='airflow',
    dbname='common_crawl_db',
)

query = """
SELECT *
FROM result_table
"""

df = pd.read_sql_query(query, conn)
df['subsections'] = df['subsections'].apply(lambda x: set(json.loads(x)))

C:\Users\hp\AppData\Local\Temp\ipykernel_14012\1828101135.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


In [13]:
df.head()

,homepage,is_homepage,count,subsections,country,category,is_ad_based
0,://,False,3,"{CC-MAIN-20240712094214-20240712124214-00000.warc.gz, CC-MAIN-20240712094214-20240712124214-00001.warc.gz, CC-MAIN-20240712094214-20240712124214-00002.warc.gz}",UNKNOWN,Food & Drink,False
1,http://0.baivli.com,False,3,{/play/nzn_cr/MYgEBA.html},UNKNOWN,Pets & Animals,False
2,http://0287327.com,False,3,{/1261520.html},UNKNOWN,Autos & Vehicles,False
3,http://029scy.com,False,3,{/bbcef/238899.html},UNKNOWN,People & Society,False
4,http://08l.happymealbox.net,False,3,{/new-equipment/attachments/couplers-loader/},UNKNOWN,Pets & Animals,False


In [14]:
# 1. Percentage of Categorised Websites
def percentage_websites_categorized(df):
    total_sites = len(df)
    categorized_sites = df[df['category'] != 'UNKNOWN']
    return round((len(categorized_sites) * 100 / total_sites), 2)

# 2. Assess how many uncategorised sites are actually ad-based
def percentage_website_not_categorized_ad_based(df):
    non_categorized_sites = df[df['category'] == 'UNKNOWN']
    non_categorized_ad_based_sites = non_categorized_sites['is_ad_based'].sum()
    return (non_categorized_ad_based_sites / len(non_categorized_sites)) * 100

# 3. Top 5 Categories per count
def top_5_categories(df):
    return df['category'].value_counts().reset_index().head(5)

# 4. Distribution of websites per country - Mode of categories per country
def mode_category_per_country(df):
    return df.groupby('country')['category'].agg(lambda x: x.mode().iloc[0])

# 5. Country with multiple sites/subdomains
def country_with_most_ws_sd(df):
    return df[df['country'] != 'UNKNOWN']['country'].value_counts().idxmax()


In [15]:
percentage_categorized_df = percentage_websites_categorized(df)
percentage_not_categorized_ad_based_df = percentage_website_not_categorized_ad_based(df)
top_5_categories_df = top_5_categories(df)
mode_per_country_df = mode_category_per_country(df)
country_with_most_ws_sd_df = country_with_most_ws_sd(df)

In [16]:
print("# 1. Percentage of Categorised Websites:", percentage_categorized_df)

# 1. Percentage of Categorised Websites: 96.38


In [17]:
print("# 2. Percentage of Uncategorized Sites that are Ad-Based:", percentage_not_categorized_ad_based_df)

# 2. Percentage of Uncategorized Sites that are Ad-Based: 50.26123301985371


In [18]:
print("Top 5 Categories per count:\n",)
top_5_categories_df

Top 5 Categories per count:



,category,count
0,Hobbies & Leisure,1969
1,Reference,1957
2,Food & Drink,1933
3,Autos & Vehicles,1933
4,Science,1927


In [19]:
print("# 4. Mode of categories per country:\n", mode_per_country_df)

# 4. Mode of categories per country:
 country
DE                          Travel
UNKNOWN                  Reference
ad               Hobbies & Leisure
ae                Autos & Vehicles
af                        Shopping
ag            Arts & Entertainment
ai                     Real Estate
al              Internet & Telecom
am                Autos & Vehicles
ao              Internet & Telecom
ar               Hobbies & Leisure
as                           Adult
at            Arts & Entertainment
au                       Reference
aw                       Reference
ax                Jobs & Education
az               Hobbies & Leisure
ba                Beauty & Fitness
bd         Computers & Electronics
be            Arts & Entertainment
bf                         Science
bg                People & Society
bh                Law & Government
bi                Law & Government
bm                          Sports
bo                         Science
br           Business & Industrial
bt       

In [20]:
print("# 5. Country with multiple sites/subdomains: ", country_with_most_ws_sd_df)

# 5. Country with multiple sites/subdomains:  ru


In [21]:
output_dir = 'output_arrow'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

table = pa.Table.from_pandas(df)

pq.write_to_dataset(
    table,
    root_path=output_dir,
    partition_cols=['category']
)